In [0]:
#Mounting google drive to access the csv files created
from google.colab import drive
#drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)
import pandas as pd

Mounted at /content/drive


In [0]:
import pickle
import pandas as pd
import numpy as np

post_info = pickle.load( open( "/content/drive/My Drive/save.p", "rb" ) )

In [0]:
post_info.drop(post_info.columns[0:1],axis=1,inplace=True)

In [0]:
post_info['likes_normalized'] = post_info['likes']/post_info['likes'].max()
post_info['comments_normalized'] = post_info['comments']/post_info['comments'].max()
post_info['engagement_score'] = .4*post_info['likes_normalized']+.6*post_info['comments_normalized']

In [0]:
engagement_median = post_info['engagement_score'].median()

def binary_engage(score):
    if score > engagement_median:
        return 1
    else:
        return 0

In [0]:
post_info['engagement_binary'] = post_info['engagement_score'].map(binary_engage)

In [0]:
post_info.engagement_binary.value_counts()

1    236
0    236
Name: engagement_binary, dtype: int64

In [0]:
regression = post_info[['labels', 'engagement_binary']]
labels = list(set(post_info['labels'].sum()))
for label in labels:
    regression[label] = regression['labels'].apply(lambda x: 1 if label in x else 0 )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [0]:
regression.head()

,labels,engagement_binary,World,Arch,Siberian tiger,Fisherman,Mist,Mouth,Maple leaf,Knee,Large-flowered cactus,Nature,Performance,Soldier,Bridge,Coyote,Basketball,Horse supplies,Photography,Shark,Perro de presa canario,Waste,Interaction,Rock,Hill,Flightless bird,Eyebrow,Cooling tower,Parrot,Fish,Cumulus,Amphibian,Sports,Companion dog,Humpback whale,Jungle,Fire,Giraffidae,Garmon,Castle,...,Stately home,Poison dart frog,Flare,Diatonic button accordion,Night,Space,Street football,Evening,Canidae,Hip,Accordionist,Lava cave,Hula hoop,Stratovolcano,Snapshot,Antler,Toad,Monochrome,Mountainous landforms,Bird of prey,Statue,Rare breed (dog),Natural environment,Cycling,Wilderness,Mare,Dugong,Surfing,Python,History,Universe,Social work,Green,Joint,Rays and skates,Nebula,Academic dress,Pony,Leisure,Water bird
0,"[Lady, Beauty, Lip, Fashion, Photo shoot, Arm,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"[Visual effect lighting, Technology, Display d...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"[Tiger, Wildlife, Vertebrate, Bengal tiger, Si...",1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"[Wave, Ocean, Surfing Equipment, Surfing, Sea,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,"[Water, Nature, Black-and-white, Atmospheric p...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import spacy

nltk.download('wordnet')

def getLemma(caption):
  nlp = spacy.load('en', disable=['parser', 'ner'])
  doc = nlp(caption)
  return " ".join([token.lemma_ for token in doc])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
post_info['caption_replaced']=post_info['caption'].map(getLemma)

In [0]:
from nltk import word_tokenize
nltk.download('punkt')
post_info['tokenized_captions'] = post_info['caption_replaced'].map(lambda x: nltk.word_tokenize(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
cv=CountVectorizer(stop_words='english', max_features=1250)
word_count_vector=cv.fit_transform(post_info.caption_replaced.values)
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
tf_idf_vector=tfidf_transformer.transform(word_count_vector)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X = tf_idf_vector
y = regression[['engagement_binary']]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train)

log_reg.score(x_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7372881355932204

In [0]:
predictions = log_reg.predict(x_test)

confusion = pd.DataFrame(zip(y_test.engagement_binary, predictions), columns=['actual', 'predicted'])
pd.crosstab(confusion.actual, confusion.predicted)

predicted,0,1
actual,,
0,45,17
1,14,42


In [0]:
label_string = pd.DataFrame(post_info.labels.map(lambda x: ' '.join(x)))
label_string.labels = label_string.labels.str.lower()

In [0]:
label_string['labels_replaced']=label_string['labels'].map(getLemma)

In [0]:
post_info['labels_replaced']=label_string['labels_replaced']
post_info['labels']=label_string['labels']

In [0]:
cv=CountVectorizer(stop_words='english', max_features=1250)
word_count_vector=cv.fit_transform(post_info.labels_replaced.values)
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
labels_tf_idf_vector=tfidf_transformer.transform(word_count_vector)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X_labels = labels_tf_idf_vector
y = regression[['engagement_binary']]

x_train, x_test, y_train, y_test = train_test_split(X_labels, y, test_size=0.25, random_state=0)
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train)

log_reg.score(x_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6949152542372882

In [0]:
predictions = log_reg.predict(x_test)

confusion = pd.DataFrame(zip(y_test.engagement_binary, predictions), columns=['actual', 'predicted'])
pd.crosstab(confusion.actual, confusion.predicted)

predicted,0,1
actual,,
0,44,18
1,18,38


In [0]:
from scipy import sparse
combined = sparse.hstack([tf_idf_vector, labels_tf_idf_vector])

In [0]:
X = combined

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train)

log_reg.score(x_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7372881355932204

In [0]:
predictions = log_reg.predict(x_test)

confusion = pd.DataFrame(zip(y_test.engagement_binary, predictions), columns=['actual', 'predicted'])
pd.crosstab(confusion.actual, confusion.predicted)

predicted,0,1
actual,,
0,48,14
1,17,39


In [0]:
post_info.head()

,display_url,caption,comments,likes,labels,likes_normalized,comments_normalized,engagement_score,engagement_binary,caption_replaced,tokenized_captions,labels_replaced
0,https://scontent-hkg3-1.cdninstagram.com/vp/56...,Photo by @dina_litovsky | These are portraits ...,365,57910,lady beauty lip fashion photo shoot arm should...,0.035820,0.028027,0.031144,0,photo by @dina_litovsky | these be portrait of...,"[photo, by, @, dina_litovsky, |, these, be, po...",lady beauty lip fashion photo shoot arm should...
1,https://scontent-hkg3-1.cdninstagram.com/vp/c3...,Photo by David Guttenfelder @dguttenfelder // ...,190,69265,visual effect lighting technology display devi...,0.042843,0.014590,0.025891,0,photo by David Guttenfelder @dguttenfelder // ...,"[photo, by, David, Guttenfelder, @, dguttenfel...",visual effect lighting technology display devi...
2,https://scontent-hkg3-1.cdninstagram.com/vp/62...,"Photo by @stevewinterphoto | Our new story, “T...",1482,421468,tiger wildlife vertebrate bengal tiger siberia...,0.260694,0.113799,0.172557,1,photo by @stevewinterphoto | -PRON- new story ...,"[photo, by, @, stevewinterphoto, |, -PRON-, ne...",tiger wildlife vertebrate bengal tiger siberia...
3,https://scontent-hkg3-1.cdninstagram.com/vp/cc...,"Photo by @davidalanharvey | Outer Banks, North...",341,160309,wave ocean surfing equipment surfing sea surfb...,0.099157,0.026184,0.055374,0,"photo by @davidalanharvey | Outer Banks , Nort...","[photo, by, @, davidalanharvey, |, Outer, Bank...",wave ocean surf equipment surf sea surfboard c...
4,https://scontent-hkg3-1.cdninstagram.com/vp/d0...,Photo by Jimmy Chin @jimmychin | Alpine dreams...,551,314242,water nature black-and-white atmospheric pheno...,0.194371,0.042310,0.103134,0,photo by Jimmy Chin @jimmychin | Alpine dreams...,"[photo, by, Jimmy, Chin, @, jimmychin, |, Alpi...",water nature black - and - white atmospheric p...


In [0]:
post_info['labels_list']=post_info['labels_replaced'].map(lambda x: nltk.word_tokenize(x))


In [0]:
post_info.to_csv('post_info.csv')

In [0]:
labels_list=post_info['labels_list'].to_list()
labels_list = [[ele for ele in sub if (ele != '-') & (ele!='and')] for sub in labels_list] 

In [0]:
from gensim import corpora, models

dictionary_LDA = corpora.Dictionary(labels_list)
dictionary_LDA.filter_extremes(no_below=3)
corpus = [dictionary_LDA.doc2bow(list_of_tokens) for list_of_tokens in labels_list]

num_topics = 5
lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_LDA)

In [0]:
topics = lda_model.show_topics()
for topic in topics:
    print (topic)

(0, '0.033*"photography" + 0.031*"sky" + 0.021*"wildlife" + 0.020*"mountain" + 0.018*"elephant" + 0.017*"water" + 0.016*"natural" + 0.016*"landscape" + 0.016*"bird" + 0.015*"landform"')
(1, '0.049*"sky" + 0.043*"water" + 0.036*"sea" + 0.027*"landscape" + 0.023*"tree" + 0.021*"natural" + 0.019*"plant" + 0.018*"turtle" + 0.017*"blue" + 0.017*"nature"')
(2, '0.031*"sky" + 0.019*"vertebrate" + 0.019*"animal" + 0.016*"wildlife" + 0.015*"mammal" + 0.015*"grassland" + 0.014*"dog" + 0.013*"carnivore" + 0.013*"event" + 0.013*"terrestrial"')
(3, '0.026*"photography" + 0.020*"people" + 0.019*"adaptation" + 0.019*"tree" + 0.017*"nature" + 0.017*"smile" + 0.016*"temple" + 0.016*"landscape" + 0.016*"event" + 0.016*"military"')
(4, '0.036*"fish" + 0.032*"shark" + 0.030*"photography" + 0.028*"white" + 0.026*"marine" + 0.025*"underwater" + 0.022*"biology" + 0.019*"sea" + 0.017*"organism" + 0.016*"art"')


In [0]:
topic_list=[]
for l in labels_list:
  bow = dictionary_LDA.doc2bow(l)
  topic_list.append(lda_model.get_document_topics(bow))


In [0]:
x=lda_model.show_topics(num_topics=5,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]
for topic,words in topics_words:
    print(str(topic)+ "::"+ str(words))
print()

0::['photography', 'sky', 'wildlife', 'mountain', 'elephant', 'water', 'natural', 'landscape', 'bird', 'landform']
1::['sky', 'water', 'sea', 'landscape', 'tree', 'natural', 'plant', 'turtle', 'blue', 'nature']
2::['sky', 'vertebrate', 'animal', 'wildlife', 'mammal', 'grassland', 'dog', 'carnivore', 'event', 'terrestrial']
3::['photography', 'people', 'adaptation', 'tree', 'nature', 'smile', 'temple', 'landscape', 'event', 'military']
4::['fish', 'shark', 'photography', 'white', 'marine', 'underwater', 'biology', 'sea', 'organism', 'art']



In [0]:
topics_words=pd.DataFrame(topics_words)
topics_words.rename(columns={0:'Topic',1:'Words'},inplace=True)

In [0]:
topics_words.to_csv('topics_words.csv')

In [0]:
topic_df.head()

,Topic1_Weight,Topic2_Weight,Topic3_Weight,Topic4_Weight,Topic5_Weight
0,0.022874,0.909769,0.022272,0.022471,0.022615
1,0.050270,0.798441,0.050238,0.050337,0.050714
2,0.014444,0.405159,0.014613,0.014349,0.551435
3,0.025302,0.025185,0.025566,0.025314,0.898634
4,0.945718,0.013524,0.013601,0.013575,0.013583


In [0]:
topic_df=topic_df[['Topic1_Weight','Topic2_Weight','Topic3_Weight','Topic4_Weight','Topic5_Weight']]

In [0]:
post_info=pd.read_csv('post_info.csv')

In [0]:
post_info=pd.concat([post_info, topic_df], axis=1)

In [0]:
post_info.to_csv('post_info.csv')

In [0]:
post_info['quartile'] = pd.qcut(post_info['engagement_score'], 4, labels=False)

In [0]:
quartile_info=post_info.groupby(['quartile'])[['Topic1_Weight','Topic2_Weight','Topic3_Weight','Topic4_Weight','Topic5_Weight']].agg('mean')

In [0]:
topic_quartile=quartile_info.T.reset_index()

In [0]:
topic_engagement=pd.concat([topic_quartile,topics_words],axis=1)

In [0]:
topic_engagement.drop(columns=['Topic'],inplace=True)
topic_engagement.rename(columns={'index':'Topic',0:'Quartile1',1:'Quartile2',2:'Quartile3',3:'Quartile4'},inplace=True)

In [0]:
topic_engagement.to_csv('topic_engagement.csv')